In [17]:
import requests
import pprint
import pandas as pd
import io

In [72]:
def get_data(country, country_pop):
    url = f'https://api.covid19api.com/dayone/country/{country}/status/confirmed/live'
    j = requests.get(url).json()

    raw_df = pd.DataFrame.from_dict(j)
    
    df = raw_df.copy()
    
    df['Date'] = df['Date'].astype('datetime64[ns]')
    df["Daily Cases"] = 0
    
    for i in range(1, len(df)):
        df.loc[i, 'Daily Cases'] = df.loc[i, 'Cases'] - df.loc[i-1, 'Cases']
    
    df['Daily Cases Per Capita'] = df['Daily Cases']/country_pop
    
    return df

In [80]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import all_palettes
output_notebook()

Loading BokehJS ...

In [90]:
class InputConfig:
    
    def __init__(self, country, base_pop):
        
        self.country = country
        self.base_pop = base_pop
        self.df = get_data(country, base_pop)
        self.data_src = ColumnDataSource(df)

In [91]:
InputConfig("italy", 60360000).df.head()

,Country,CountryCode,Province,City,CityCode,Lat,Lon,Cases,Status,Date,Daily Cases,Daily Cases Per Capita
0,Italy,IT,,,,41.87,12.57,2,confirmed,2020-01-31,0,0.0
1,Italy,IT,,,,41.87,12.57,2,confirmed,2020-02-01,0,0.0
2,Italy,IT,,,,41.87,12.57,2,confirmed,2020-02-02,0,0.0
3,Italy,IT,,,,41.87,12.57,2,confirmed,2020-02-03,0,0.0
4,Italy,IT,,,,41.87,12.57,2,confirmed,2020-02-04,0,0.0


In [86]:
input_configs = [
    InputConfig("germany", 83020000),
    InputConfig("italy", 60360000),
]

In [88]:
p = figure(plot_width=400, plot_height=400, x_axis_type='datetime')

all_palettes['Colorblind'][8]

for input_config, color in zip(input_configs, all_palettes['Colorblind'][len(input_configs) +1]):

    p.circle(
        x="Date",
        y="Daily Cases Per Capita",
        fill_color=color,
        legend_label=input_config.country.title(),
        source=input_config.data_src
    )

show(p)